In [24]:
import tweepy 

from os import getenv
import psycopg2
from dotenv import load_dotenv
load_dotenv()

TWITTER_CONSUMER_API_KEY = getenv('TWITTER_CONSUMER_API_KEY')
TWITTER_CONSUMER_API_SECRET = getenv('TWITTER_CONSUMER_API_SECRET')
TWITTER_ACCESS_TOKEN = getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_TOKEN_SECRET = getenv('TWITTER_ACCESS_TOKEN_SECRET')

TWITTER_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY,TWITTER_CONSUMER_API_SECRET)
TWITTER_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
TWITTER = tweepy.API(TWITTER_AUTH)

In [34]:
twitter_user = TWITTER.user_timeline(screen_name="elonmusk", max_id=1491529818986487816)


In [37]:
len(twitter_user)

20

In [33]:
twitter_user[1].id


1491529818986487816

In [16]:




def connect(DATABASE, USERNAME, PASSWORD, HOST):
    """ Connect to the PostgreSQL database server """
    elephantsql_client = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')

        # Connect to ElephantSQL-hosted PostgreSQL
        elephantsql_client = psycopg2.connect(dbname=DATABASE, user=USERNAME, password=PASSWORD, host=HOST)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    return elephantsql_client

ELEPHANTSQL_DATABASE = getenv('ELEPHANTSQL_DATABASE')
ELEPHANTSQL_USERNAME = getenv('ELEPHANTSQL_USERNAME')
ELEPHANTSQL_PASSWORD = getenv('ELEPHANTSQL_PASSWORD')
ELEPHANTSQL_HOST = getenv('ELEPHANTSQL_HOST')


# Building Inital Table
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

# cur = elephantsql_client.cursor()
# cur.close()

agkecyit
Connecting to the PostgreSQL database...


In [ ]:
def create_table(elephantsql_client):
    ''' Creating a table based on whats being stored on twitoff app'''

    # For the purposes of this I will handwrite the sql commands
    command = '''
        CREATE TABLE IF NOT EXISTS usernames_table (id                      SERIAL PRIMARY KEY,
                                                    profile_image_url       varchar(30)
                                                    username                varchar(30),
                                                    real_name               varchar(30),
                                                    bio                     varchar(200),
                                                    birthday                varchar(30),
                                                    followers               varchar(30),
                                                    following               varchar(30))
                                                
        '''

    command = '''
    CREATE TABLE IF NOT EXISTS recent_comparisons_table (id                 SERIAL PRIMARY KEY,
                                                         result             varchar(50),
                                                         user_one           varchar(30),
                                                         user_two           varchar(30),
                                                         input_text         varchar(500))
                                            
    '''

    command = '''
    CREATE TABLE IF NOT EXISTS {}_tweets_table (id                 SERIAL PRIMARY KEY,
                                                tweet_text             varchar(500))
                                            
    '''
    
    try:

        # A "cursor", a structure to iterate over db records to perform queries
        cur = elephantsql_client.cursor()

        # Execute commands in order
        cur.execute(command)

        # Close communication with the PostgreSQL database server
        cur.close()

        # Commit the changes
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

def single_insert(elephantsql_client, insert_req):
    """ Execute a single INSERT request """
    cur = elephantsql_client.cursor()
    try:
        cur.execute(insert_req)
        elephantsql_client.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 1
    cur.close()